In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision.ops
from torch import nn

class DeformableConv2d(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=1,
                 dilation=1,
                 bias=False):
        super(DeformableConv2d, self).__init__()

        assert type(kernel_size) == tuple or type(kernel_size) == int

        kernel_size = kernel_size if type(kernel_size) == tuple else (kernel_size, kernel_size)
        self.stride = stride if type(stride) == tuple else (stride, stride)
        self.padding = padding
        self.dilation = dilation

        self.offset_conv = nn.Conv2d(in_channels,
                                     2 * kernel_size[0] * kernel_size[1],
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=self.padding,
                                     dilation=self.dilation,
                                     bias=True)

        nn.init.constant_(self.offset_conv.weight, 0.)
        nn.init.constant_(self.offset_conv.bias, 0.)

        self.modulator_conv = nn.Conv2d(in_channels,
                                        1 * kernel_size[0] * kernel_size[1],
                                        kernel_size=kernel_size,
                                        stride=stride,
                                        padding=self.padding,
                                        dilation=self.dilation,
                                        bias=True)

        nn.init.constant_(self.modulator_conv.weight, 0.)
        nn.init.constant_(self.modulator_conv.bias, 0.)

        self.regular_conv = nn.Conv2d(in_channels=in_channels,
                                      out_channels=out_channels,
                                      kernel_size=kernel_size,
                                      stride=stride,
                                      padding=self.padding,
                                      dilation=self.dilation,
                                      bias=bias)

    def forward(self, x):
        # h, w = x.shape[2:]
        # max_offset = max(h, w)/4.

        offset = self.offset_conv(x)  # .clamp(-max_offset, max_offset)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x))
        # op = (n - (k * d - 1) + 2p / s)
        x = torchvision.ops.deform_conv2d(input=x,
                                          offset=offset,
                                          weight=self.regular_conv.weight,
                                          bias=self.regular_conv.bias,
                                          padding=self.padding,
                                          mask=modulator,
                                          stride=self.stride,
                                          dilation=self.dilation)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Paper_model(nn.Module):
    def __init__(self,classes=8, conv_num=32):
        super(Paper_model, self).__init__()
        bn_axis = 1  # assuming channels first
        self.conv_num = conv_num

        self.conv_1_offset = DeformableConv2d(3,conv_num)
        self.conv_1 = nn.Conv2d(conv_num, conv_num, kernel_size=3, stride=2, padding=1)
        self.batch_normalization_1 = nn.BatchNorm2d(conv_num)

        self.conv_2_offset = DeformableConv2d(conv_num,conv_num*2)
        self.conv_2 = nn.Conv2d(conv_num * 2 , conv_num * 2, kernel_size=3, stride=2, padding=1)
        self.batch_normalization_2 = nn.BatchNorm2d(conv_num * 2)

        self.conv_3_offset = DeformableConv2d(conv_num * 2,conv_num * 4)
        self.conv_3 = nn.Conv2d(conv_num * 4, conv_num * 4, kernel_size=3, stride=2, padding=1)
        self.batch_normalization_3 = nn.BatchNorm2d(conv_num * 4)

        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(conv_num * 4, classes)

    def forward(self, x):
        x = self.conv_1_offset(x)
        x = self.conv_1(x)
        x = self.batch_normalization_1(x)
        x = F.gelu(x)

        x = self.conv_2_offset(x)
        x = self.conv_2(x)
        x = self.batch_normalization_2(x)
        x = F.gelu(x)

        x = self.conv_3_offset(x)
        x = self.conv_3(x)
        x = self.batch_normalization_3(x)
        x = F.gelu(x)

        x = self.pooling(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.sigmoid(x)


In [ ]:
you = DeformableConv2d(1,50)
print(you.forward(torch.randn(1,1,52,52)).shape)
x = torch.randn(10,3,52,52)
print(type(x))
test = Paper_model()
print(test.forward(x).shape)
print(test)

torch.Size([1, 50, 52, 52])
<class 'torch.Tensor'>
torch.Size([10, 8])
Paper_model(
  (conv_1_offset): DeformableConv2d(
    (offset_conv): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (modulator_conv): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (regular_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (conv_1): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (batch_normalization_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_2_offset): DeformableConv2d(
    (offset_conv): Conv2d(32, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (modulator_conv): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (regular_conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (conv_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (batch_normalization_2): BatchNo

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

class WaferDataset(Dataset):
    def __init__(self, img_array, label_array):
        self.img_array = img_array
        self.label_array = label_array

    def __len__(self):
        return len(self.img_array)

    def __getitem__(self, idx):
        img = torch.from_numpy(self.img_array[idx]).float()
        if img.ndim == 2:  # If the image is grayscale
            img = img.unsqueeze(0).repeat(3, 1, 1)  # Repeat the single channel 3 times
        elif img.ndim == 3:  # If the image is already multi-channel but in [H, W, C] format
            img = img.permute(2, 0, 1)  # Rearrange from [H, W, C] to [C, H, W]
        label = torch.from_numpy(self.label_array[idx]).float()
        return img, label

# 加载数据
data = np.load('/content/drive/MyDrive/DeepLearning_project/Dataset/images_3chnl.npz') # V3_dataset.npz
img_array = data['arr_0']
label_array = data['arr_1']

# 创建数据集
wafer_dataset = WaferDataset(img_array, label_array)

# 设置划分比例
val_split = 0.2
dataset_size = len(wafer_dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)
val_size = int(np.floor(val_split * dataset_size))
train_indices, val_indices = indices[val_size:], indices[:val_size]

# 创建训练集和验证集的 SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# 创建数据加载器
batch_size = 32
train_dataloader = DataLoader(wafer_dataset, batch_size=batch_size, sampler=train_sampler)
val_dataloader = DataLoader(wafer_dataset, batch_size=batch_size, sampler=val_sampler)


In [ ]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 创建数据集
wafer_dataset = WaferDataset(img_array, label_array)

# 创建数据加载器
batch_size = 32
dataloader = DataLoader(wafer_dataset, batch_size=batch_size, shuffle=True)

# 使用数据加载器进行训练
# 创建模型实例
paper_model = Paper_model()

# 定义损失函数
criterion = nn.BCELoss()

# 定义优化器
optimizer = optim.Adam(paper_model.parameters(), lr=0.001)  # 可以调整学习率


num_epochs = 30
# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 将模型移动到设备上
paper_model.to(device)

# 加载数据集的代码

# 创建数据加载器，并将数据移动到设备上
dataloader = DataLoader(wafer_dataset, batch_size=batch_size, shuffle=True)
dataloader = [(imgs.to(device), labels.to(device)) for imgs, labels in dataloader]

import random
from tqdm import tqdm
def caculate_acc_num(outputs,labels):
  acc_num = 0
  predicted = (outputs > 0.5).float()
  #print(outputs.shape)
  for idx in range(predicted.shape[0]):
    #print("Predicted : ",predicted[idx])
    #print("Labels : ",labels[idx])
    if torch.allclose(predicted[idx], labels[idx]):
      acc_num += 1

  return acc_num
# 使用数据加载器进行训练
for epoch in range(num_epochs):
    train_acc_num = 0
    total_train = 0
    train_loss = 0
    for imgs, labels in tqdm(train_dataloader):
        # 清除梯度
        optimizer.zero_grad()
        imgs = imgs.to(device)
        labels = labels.to(device)
        # 前向传播
        outputs = paper_model(imgs)
        total_train += imgs.shape[0]
        train_acc_num += caculate_acc_num(outputs,labels)
        # 计算损失
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()

        # 更新权重
        optimizer.step()
        # 随机选择一笔数据打印其标签和预测值
        #idx = random.randint(0, len(labels) - 1)
        #print(f'Label: {labels[idx]}, Prediction: {(outputs[idx] > 0.5).float()}')

    val_acc_num = 0
    total_val = 0
    val_loss = 0
    for imgs, labels in tqdm(val_dataloader):
        # 前向传播
        imgs = imgs.to(device)
        labels = labels.to(device)
        outputs = paper_model(imgs)
        total_val += imgs.shape[0]
        val_acc_num += caculate_acc_num(outputs, labels)
        # 计算损失
        loss = criterion(outputs, labels)
        val_loss += loss.item()


    # 每个 epoch 结束后打印损失
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {(train_loss/total_train):.4f} , Acc : {(train_acc_num/total_train):.4f}')
    print(f'Val_loss: {(val_loss/total_val):.4f}, Val_Acc: {(val_acc_num/total_val):.4f}')


Using device: cuda


100%|██████████| 238/238 [00:02<00:00, 98.10it/s]


Epoch [1/30], Loss: 0.0042 , Acc : 0.7198
Val_loss: 0.0015, Val_Acc: 0.9090


100%|██████████| 238/238 [00:02<00:00, 86.73it/s] 


Epoch [2/30], Loss: 0.0012 , Acc : 0.9227
Val_loss: 0.0011, Val_Acc: 0.9332


...

Epoch [28/30], Loss: 0.0002 , Acc : 0.9876
Val_loss: 0.0005, Val_Acc: 0.9720


100%|██████████| 238/238 [00:02<00:00, 87.16it/s]


Epoch [29/30], Loss: 0.0002 , Acc : 0.9886
Val_loss: 0.0005, Val_Acc: 0.9734


100%|██████████| 238/238 [00:02<00:00, 95.21it/s]

Epoch [30/30], Loss: 0.0001 , Acc : 0.9903
Val_loss: 0.0005, Val_Acc: 0.9722
